# Programming Assignment 2 - Multiple Linear Regression (80 points)

## Implementing Multiple Linear Regression Using Gradient Descent And Stochastic Gradient Descent Algorithm.

In this assignment, you will be implementing Multiple Linear Regression Model on the same data set as in Programming Assignment 1, but here we will be using gradient descent algorithm (GDA) and stochastic gradient descent algorithm (SGDA) to minimize the cost function as taught in the class. The SGDA implementation is optional.

Please add your own print statements to check your code to ensure your code is correct in every step.  (Note: we will not be grading the print statements you add to your code.)

In [1]:
# Run this code cell using Shift + Enter before moving further
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt

# Loading the data set.

In the below code cell, you will load the data using python pandas library as done in the programming_assignment_1b. Use pd.read_csv('File url ', header=None,.... ) with the value of header=None,delim_whitespace=True,names=names,na_values='?' as attributes. The url for the .data file is https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data. This step is same as done in programming_assignment_1b.

In [2]:
# After completing the code in this code cell, run this code cell before moving further.
names =[
    'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 
    'AGE',  'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE'
]

# TODO - write the pandas command to read the csv file into a dataframe df - 5 points
df= pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data", header=None, names=names, delim_whitespace=True, na_values='?')
print(df)


         CRIM    ZN  INDUS  CHAS    NOX     RM    AGE     DIS  RAD    TAX  \
0     0.00632  18.0   2.31     0  0.538  6.575   65.2  4.0900    1  296.0   
1     0.02731   0.0   7.07     0  0.469  6.421   78.9  4.9671    2  242.0   
2     0.02729   0.0   7.07     0  0.469  7.185   61.1  4.9671    2  242.0   
3     0.03237   0.0   2.18     0  0.458  6.998   45.8  6.0622    3  222.0   
4     0.06905   0.0   2.18     0  0.458  7.147   54.2  6.0622    3  222.0   
5     0.02985   0.0   2.18     0  0.458  6.430   58.7  6.0622    3  222.0   
6     0.08829  12.5   7.87     0  0.524  6.012   66.6  5.5605    5  311.0   
7     0.14455  12.5   7.87     0  0.524  6.172   96.1  5.9505    5  311.0   
8     0.21124  12.5   7.87     0  0.524  5.631  100.0  6.0821    5  311.0   
9     0.17004  12.5   7.87     0  0.524  6.004   85.9  6.5921    5  311.0   
10    0.22489  12.5   7.87     0  0.524  6.377   94.3  6.3467    5  311.0   
11    0.11747  12.5   7.87     0  0.524  6.009   82.9  6.2267    5  311.0   

Now, instead of taking just 1 feature i.e 'RM' as we did in Simple Linear Regression, we will use 4 features which are 'LSTAT', 'DIS', 'RM' and 'INDUS'. We choose only these features because they are almost on similar scale. So, there is no need for feature scaling. Fetching the values of 5 columns into a smaller dataframe (say) df1 from df. This step is also same as done in programming_assignment_1b.There we fetched only 2 columns but here we need 5 columns.We need the values in the 'PRICE', 'LSTAT', 'DIS', 'RM' and 'INDUS' columns.

In [3]:
# TODO - Save values from columns 'PRICE', 'LSTAT', 'DIS', 'RM', 'INDUS' into df1 from df - 5 points
df1= df[['PRICE', 'LSTAT', 'DIS', 'RM', 'INDUS']]

# TODO - Remove nan values from df1 and save into df2 - 5 points
df2=df1[~np.isnan(df1)]

# Check the shape of df2. It should be (506,5)
print(df2.shape)


(506, 5)


In [4]:
# TODO - Create a vector y having the values of 'PRICE' column and matrix x having the values of
# LSTAT', 'DIS', 'RM' and 'INDUS' columns - 5 points

df3 = df2[['LSTAT', 'DIS', 'RM', 'INDUS']]
x=df3.values
y=np.array(df2['PRICE'])

# Check the shape of x and y vectors.
print(x.shape)
print(y.shape)

(506, 4)
(506,)


Reshape y to be rank 2. After checking the shape of x and y in the above code cell, we see that x is already rank 2 and y are rank 1 matrix. Before moving ahead, convert y to be rank 2 matrix. For example, I would use the command y=y.reshape(y.shape[0],1) to reshape.

In [5]:
# TODO - Reshape y into a rank 2 matrix - 5 points
y = y.reshape(y.shape[0], 1)
print(y.shape)

(506, 1)


Calculating the value of n i.e. number of training examples. 
Hint: Value of n is equal to the number of rows in either x or y matrix which can be accessed using numpy shape command.

In [6]:
# TODO - Save number of training examples into n and print it - 5 points
n = x.shape[0]
print(n)

506


In [7]:
# TODO - Append a column of ones in x - 5 points

a=np.ones((y.shape[0],1))
x=np.hstack((a , x))
print(x)

# Shape of x should be (506,5)
print(x.shape)

[[ 1.      4.98    4.09    6.575   2.31  ]
 [ 1.      9.14    4.9671  6.421   7.07  ]
 [ 1.      4.03    4.9671  7.185   7.07  ]
 ...
 [ 1.      5.64    2.1675  6.976  11.93  ]
 [ 1.      6.48    2.3889  6.794  11.93  ]
 [ 1.      7.88    2.505   6.03   11.93  ]]
(506, 5)


# Cost Function.
Compute the cost: Write the code to compute the cost inside the function. Do not change the function name or function parameters.

In [8]:
def compute_cost(x, y, w, n):
    # Write your code in place of None. Cost can be calculated using a single line of code.
    # Remember w is a vector here.
    
    # TODO - Write the cost function - 10 points
    cost=np.sum((y-np.dot(x,w))**2)/(2*n)
    
    return cost

Before moving ahead, ensure that the code you have written to compute the cost is correct. Just run the below cell.

In [9]:
w_testcase=np.zeros((5,1))
cost_verify= compute_cost(x, y, w_testcase, n)
print(cost_verify)

296.0734584980237


Your output should be equal to 296.073458498. If it's equal to this, then move ahead. Else, re-check your code and re-verify.

# Gradient Descent
Write the code to perform gradient descent in the function below

In [10]:
def gradient_descent(x , y , learning_rate , w , n , num_iters):
    # In place of None, write the updated value of w0 in temp0 and of w1 in temp1
    # TODO - Finish the gradient descent function - 10 points
    
    for i in range(num_iters):
        # derivative vector is given by : X_train.Transpose *  (( X_train * w)- y ) 
        temp =  np.dot(x.T, np.dot(x,w)-y)/n
        w = w-learning_rate*temp
       
        
        if(i%100==0):
            # In place of None, call the cost you just coded above
            cost= compute_cost(x,y,w,n)
            print("Cost")
            print(cost)
             
    return w      

Before moving ahead, ensure that your code to update w is correct. Run the code cell below.

In [11]:
w_testcase=np.zeros((5,1))
g=gradient_descent(x , y , 0.0049 , w_testcase , n , 100000)
print(g[0])

Cost
267.23946864871743
Cost
17.992016338677416
Cost
15.293488231121279
Cost
14.678638998273042
Cost
14.527249342133468
Cost
14.489625150767532
Cost
14.479943796087383
Cost
14.477127192952596
Cost
14.475998525491512
Cost
14.475286351846949
Cost
14.474678395546796
Cost
14.474097952523573
Cost
14.473526175661238
Cost
14.472958427629937
Cost
14.47239356108069
Cost
14.471831285845981
Cost
14.471271522320357
Cost
14.470714242649628
Cost
14.470159431726556
Cost
14.46960707761116
Cost
14.469057169177788
Cost
14.468509695537064
Cost
14.467964645893751
Cost
14.467422009511695
Cost
14.466881775705044
Cost
14.466343933835946
Cost
14.465808473313809
Cost
14.465275383594975
Cost
14.464744654182478
Cost
14.464216274625837
Cost
14.463690234520833
Cost
14.463166523509317
Cost
14.46264513127901
Cost
14.46212604756327
Cost
14.46160926214094
Cost
14.461094764836094
Cost
14.460582545517871
Cost
14.46007259410027
Cost
14.459564900541933
Cost
14.459059454845981
Cost
14.458556247059777
Cost
14.45805526727476

Cost
14.36350939849214
Cost
14.3634292478305
Cost
14.363349452043673
Cost
14.363270009560416
Cost
14.363190918816446
Cost
14.3631121782544
Cost
14.363033786323822
Cost
14.362955741481098
Cost
14.362878042189472
Cost
14.362800686918986
Cost
14.362723674146444
Cost
14.36264700235541
Cost
14.362570670036149
Cost
14.362494675685626
Cost
14.362419017807445
Cost
14.362343694911846
Cost
14.362268705515659
Cost
14.362194048142282
Cost
14.362119721321653
Cost
14.362045723590217
Cost
14.361972053490902
Cost
14.361898709573081
Cost
14.36182569039256
Cost
14.361752994511528
Cost
14.361680620498547
Cost
14.361608566928517
Cost
14.361536832382638
Cost
14.36146541544841
Cost
14.361394314719572
Cost
14.361323528796094
Cost
14.361253056284143
Cost
14.36118289579606
Cost
14.361113045950333
Cost
14.361043505371558
Cost
14.360974272690429
Cost
14.3609053465437
Cost
14.360836725574156
Cost
14.360768408430598
Cost
14.360700393767804
Cost
14.360632680246518
Cost
14.360565266533403
Cost
14.360498151301028
Cos

The last output should be: 7.46236374. If it matches your out, then your code is likely to be correct. Otherwise, re-check your code and re-verify.

# Integrating the Batch Gradient Descent Function 

Integrating the above function into a single function multiple_linear_reg_model_gda: This function uses gradient descent algorithm to minimize the cost.

In [12]:
def multiple_linear_reg_model_gda(x , y , n , learning_rate , num_iters):
    #initialize the values of parameter vector w. It should be a column vector of zeros of dimension(n,1)
    # TODO - Complete the function - 15 points
    w = np.zeros((x.shape[1],1))
    
    # Calculate the initial cost by calling the function you coded above.
    initial_cost= compute_cost(x, y, w, n)
    print("Initial Cost")
    print(initial_cost)
    
    # Calculate the optimized value of gradients by calling the gradient_descent function coded above
    
    w = gradient_descent(x , y , learning_rate , w , n , num_iters)
    
    # Calculate the cost with the optimized value of w0 and w1 by calling the cost function.
    
    final_cost = compute_cost(x, y, w, n)
    print("Final Cost")
    print(final_cost)
    return w

Now, when you have coded multiple_linear_reg_model_gda function, you can call this function to find the optimized values of parameters w. Before calling the function, set the values of learning_rate and num_iters. You may have to call this function several number of times with different values of num_iters and learning_rate to find the optimal values of w. For some values of learning_rate, it may give an error as the values of cost may reach a very high value(infinity) due to overshooting as discussed in the class. 

In [13]:
learning_rate = 0.004
num_iters = 200000

# In place of None, call the multiple_linear_reg_model_gda.
w = multiple_linear_reg_model_gda(x, y, n, learning_rate, num_iters)
print(w)

# The value of final cost should be 14.3470049896 or nearly this(depending on the values of learning_rate and num_itersations you choose.)

Initial Cost
296.0734584980237
Cost
177.91029644311985
Cost
18.783155533825084
Cost
15.848324982705702
Cost
14.914464879849719
Cost
14.617083861003353
Cost
14.522161963317798
Cost
14.491641568176327
Cost
14.481607858388788
Cost
14.47809194480921
Cost
14.476650452657358
Cost
14.475869982517624
Cost
14.475300941280015
Cost
14.47480031345131
Cost
14.474322602971977
Cost
14.473853333800216
Cost
14.473387897081393
Cost
14.472924822634125
Cost
14.472463638674094
Cost
14.47200419101252
Cost
14.47154642649962
Cost
14.471090324140004
Cost
14.470635873181063
Cost
14.47018306614306
Cost
14.46973189660199
Cost
14.469282358484318
Cost
14.468834445842532
Cost
14.46838815278375
Cost
14.467943473446917
Cost
14.467500401995531
Cost
14.467058932615256
Cost
14.466619059513114
Cost
14.466180776917192
Cost
14.465744079076464
Cost
14.465308960260728
Cost
14.464875414760492
Cost
14.464443436886922
Cost
14.46401302097174
Cost
14.463584161367173
Cost
14.46315685244587
Cost
14.462731088600812
Cost
14.4623068642

14.369154215975117
Cost
14.369068349647335
Cost
14.368982793797567
Cost
14.368897547303176
Cost
14.368812609045584
Cost
14.368727977910261
Cost
14.368643652786703
Cost
14.36855963256842
Cost
14.368475916152937
Cost
14.368392502441745
Cost
14.368309390340325
Cost
14.368226578758096
Cost
14.368144066608444
Cost
14.368061852808667
Cost
14.367979936279978
Cost
14.367898315947505
Cost
14.367816990740247
Cost
14.367735959591078
Cost
14.367655221436744
Cost
14.367574775217815
Cost
14.367494619878709
Cost
14.36741475436765
Cost
14.367335177636676
Cost
14.367255888641598
Cost
14.36717688634202
Cost
14.367098169701299
Cost
14.367019737686537
Cost
14.36694158926857
Cost
14.366863723421979
Cost
14.366786139125017
Cost
14.366708835359658
Cost
14.366631811111535
Cost
14.366555065369974
Cost
14.366478597127943
Cost
14.366402405382042
Cost
14.366326489132513
Cost
14.366250847383208
Cost
14.36617547914158
Cost
14.366100383418672
Cost
14.366025559229108
Cost
14.365951005591059
Cost
14.365876721526266
Co

Cost
14.347565463686777
Cost
14.347557658608975
Cost
14.347549881753004
Cost
14.347542133016814
Cost
14.34753441229874
Cost
14.347526719497464
Cost
14.347519054512047
Cost
14.347511417241911
Cost
14.347503807586845
Cost
14.347496225446994
Cost
14.347488670722868
Cost
14.347481143315338
Cost
14.34747364312563
Cost
14.34746617005533
Cost
14.347458724006378
Cost
14.347451304881073
Cost
14.347443912582062
Cost
14.347436547012341
Cost
14.347429208075267
Cost
14.34742189567454
Cost
14.347414609714203
Cost
14.347407350098665
Cost
14.347400116732656
Cost
14.347392909521266
Cost
14.347385728369924
Cost
14.347378573184404
Cost
14.347371443870813
Cost
14.347364340335606
Cost
14.347357262485573
Cost
14.34735021022784
Cost
14.347343183469869
Cost
14.347336182119456
Cost
14.347329206084733
Cost
14.347322255274166
Cost
14.347315329596546
Cost
14.347308428960991
Cost
14.347301553276962
Cost
14.347294702454237
Cost
14.347287876402914
Cost
14.347281075033434
Cost
14.347274298256547
Cost
14.3472675459833

Cost
14.345904650774377
Cost
14.345902850913093
Cost
14.345901057559805
Cost
14.345899270690973
Cost
14.345897490283152
Cost
14.345895716312986
Cost
14.345893948757192
Cost
14.345892187592584
Cost
14.345890432796041
Cost
14.345888684344548
Cost
14.345886942215154
Cost
14.345885206385006
Cost
14.345883476831322
Cost
14.345881753531408
Cost
14.345880036462656
Cost
14.345878325602529
Cost
14.345876620928584
Cost
14.345874922418448
Cost
14.345873230049834
Cost
14.345871543800536
Cost
14.345869863648433
Cost
14.345868189571469
Cost
14.345866521547686
Cost
14.345864859555192
Cost
14.34586320357218
Cost
14.345861553576915
Cost
14.345859909547755
Cost
14.345858271463124
Cost
14.345856639301529
Cost
14.345855013041552
Cost
14.345853392661857
Cost
14.345851778141173
Cost
14.345850169458323
Cost
14.3458485665922
Cost
14.345846969521762
Cost
14.34584537822606
Cost
14.345843792684214
Cost
14.345842212875416
Cost
14.345840638778936
Cost
14.345839070374126
Cost
14.345837507640397
Cost
14.345835950557

# Normal Equation Method
Now, we will be writing the code to find the values of parameters w for our multiple linear regression model. This can also be used to cross-check the optimal values of w we just found above using method above. These values should be same (or nearly same).
Instead of writing the code for normal equation in one line, you can break this into 3 parts: First calculate q=inverse of (dot of (X.T,X)) (these are pseudo commands, use original numpy commands to calculate q). Then z= dot of ( X.T , y) and then w_vec= dot of (q,z). Here, w_vec is vector of dimension (5,1) having two values. Example w0=w_vec[0][0].

Note : Do not append a column of ones in x because you have already done before. 

In [14]:
# Write the code below
# TODO - Write the normal equation method - 15 points

q = np.linalg.inv(np.dot(x.T, x))
z = np.dot(x.T,y)
w_vec = np.dot(q, z)


In [15]:
print(w_vec)

[[ 8.29647186]
 [-0.66467285]
 [-0.90387731]
 [ 4.58064345]
 [-0.24342177]]


The values of w you just got above should be approximately same as the ones you got using multiple_linear_reg_model_gda.
This assignment ends here.